#### Concept

The idea of Givens rotation is to `individually` turn subdiagonal entries into zeros using a series of orthogonal matrices and thereby doing QR decomposition

This procedure is illustrated using a matrix $A\in \mathbf{R}^{3 \times 3}$ where `x` indicates entries that are not necessarily zero

$$A:\begin{bmatrix} \times & \times & \times \\\times & \times & \times\\\times & \times & \times\end{bmatrix} \rightarrow G_1A\rightarrow \begin{bmatrix} \times & \times & \times \\ 0& \times & \times\\ \times & \times & \times\end{bmatrix}\rightarrow G_2G_1A\rightarrow\begin{bmatrix} \times & \times & \times \\ 0& \times & \times\\ 0 & \times & \times\end{bmatrix}\rightarrow G_3G_2G_1A\rightarrow \begin{bmatrix} \times & \times & \times \\ 0& \times & \times\\ 0 & 0 & \times\end{bmatrix}$$

For a dense matrix of size $n$, the method of Givens rotation requires up to $\frac{1}{2}n(n-1)$ transformations. However, when a matrix is sparse, the method can be more efficient than other QR methods

#### Structure of Givens rotation matrices

Each orthogonal matrix in Givens rotation would essentially be an identity matrix except for two rows and two columns

$$G=\begin{bmatrix}
1 & & & & & & & & & & \\
& \ddots & & & & & & & & & \\
& & 1 & & & & & & & & \\
& & & c & & & & s & & & \\
& & & & 1 & & & & & & \\
& & & & & \ddots & & & & & \\
& & & & & & 1 & & & & \\
& & & -s & & & & c & & & \\
& & & & & & & & 1 & & \\
& & & & & & & & & \ddots & \\
& & & & & & & & & & 1
\end{bmatrix}$$

If $c^2+s^2=1$, we can verify that $G$ is orthogonal

#### Sequential transformtion

For a matrix of interest $A\in \mathbf{R}^{n \times n}$, Givens rotation work column-wise and for each column the method starts from the highest non-zero subdiagonal entry

For example

$$a_1=\begin{bmatrix}a_{11} \\ a_{21} \\ a_{31} \\ \vdots \\ a_{n1} \end{bmatrix}\rightarrow G_1a_1\rightarrow \begin{bmatrix}\alpha \\ 0 \\ a_{31} \\ \vdots \\ a_{n1} \end{bmatrix}$$

Since $G_1$ is orthogonal, then the norm of $a_1$ before and after the transformation is preserved and we have $\alpha = \sqrt{a_{11}^2+a_{21}^2}$

For this case, we know the rotation matrix should have the following form

$$G_1=\begin{bmatrix}c & s & & & \\ -s & c & & & \\ & & 1 & & \\
& & & \ddots & \\ & & & & 1\end{bmatrix}$$

and we can work out that

$$c=\frac{a_{11}}{\alpha}, s=\frac{a_{21}}{\alpha}$$

#### Example

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [ ]:
def givens_rotation(A):
    m, n = A.shape
    Q = np.identity(m)
    for col in range(n):
        for row in range(col+1, m):
            if A[row, col] != 0:
                alpha = np.sqrt(A[col, col]**2 + A[row, col]**2)
                c = A[col, col] / alpha
                s = A[row, col] / alpha

                G = np.identity(m)
                G[row, col] = -s
                G[col, row] = s
                G[col, col] = c
                G[row, row] = c

                A = G @ A
                Q = Q @ G.T

    return Q, A

In [ ]:
mat_list = ['square', 'tall', 'fat', 'ill-conditioned', 'identity']
mat = mat_list[2]
epsilon = 1e-8

if mat == 'square':
    A = np.array([[1.0, 2.0, 3.0, 4.0],
                  [4.0, 1.0, 0.0, -1.0],
                  [3.0, 5.0, -2.0, 1.0],
                  [2.0, 0.0, 1.0, 2.0]])
elif mat == 'tall':
    A = np.array([[1.0, 2.0, 1.0],
                  [4.0, 1.0, 4.0],
                  [3.0, 5.0, 3.0],
                  [2.0, 0.0, 1.0]])
elif mat == 'fat':
    A = np.array([[1.0, 2.0, 3.0, 3.0, 1.0],
                  [4.0, 1.0, 0.0, -0.0, 1.0],
                  [3.0, 5.0, -2.0, -2.0, 2.0],
                  [2.0, 0.0, 1.0, 1.0, 5.0]])
elif mat == 'ill-conditioned':
    A = np.array([[1, 1, 1],
                  [epsilon, 0, 0],
                  [0, epsilon, 0],
                  [0, 0, epsilon]])
elif mat == 'identity':
    A = np.identity(4)

Q, R = givens_rotation(A)

print("Orthonormal basis Q:")
print(Q)

print("\nUpper triangular matrix R:")
print(R)

print(f"\nQ^TQ:\n{np.dot(Q.T, Q)}")
print(f"Norms: \n{np.linalg.norm(Q, axis=0)}")

# Reconstruct A
A_reconstructed = Q @ R
print(f"\nA_reconstructed:\n{A_reconstructed}")

print(f"\nOriginal A:\n{A}")

Orthonormal basis Q:
[[ 0.1826  0.3324  0.9102 -0.1667]
 [ 0.7303 -0.4602 -0.0700 -0.5000]
 [ 0.5477  0.7414 -0.3501  0.1667]
 [ 0.3651 -0.3579  0.2100  0.8333]]

Upper triangular matrix R:
[[ 5.4772  3.8341 -0.1826 -0.1826  3.8341]
 [ 0.0000  3.9115 -0.8437 -0.8437 -0.4346]
 [-0.0000 -0.0000  3.6407  3.6407  1.1902]
 [ 0.0000  0.0000 -0.0000 -0.0000  3.8333]]

Q^TQ:
[[ 1.0000  0.0000 -0.0000 -0.0000]
 [ 0.0000  1.0000 -0.0000  0.0000]
 [-0.0000 -0.0000  1.0000 -0.0000]
 [-0.0000  0.0000 -0.0000  1.0000]]
Norms: 
[ 1.0000  1.0000  1.0000  1.0000]

A_reconstructed:
[[ 1.0000  2.0000  3.0000  3.0000  1.0000]
 [ 4.0000  1.0000  0.0000  0.0000  1.0000]
 [ 3.0000  5.0000 -2.0000 -2.0000  2.0000]
 [ 2.0000  0.0000  1.0000  1.0000  5.0000]]

Original A:
[[ 1.0000  2.0000  3.0000  3.0000  1.0000]
 [ 4.0000  1.0000  0.0000 -0.0000  1.0000]
 [ 3.0000  5.0000 -2.0000 -2.0000  2.0000]
 [ 2.0000  0.0000  1.0000  1.0000  5.0000]]
